In [16]:
!pip install -q sudachipy sudachidict_core

     |████████████████████████████████| 70 kB 5.4 MB/s 
     |████████████████████████████████| 473 kB 18.6 MB/s 


In [17]:
pip install torchtext==0.6.0

     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 1.2 MB 26.3 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [18]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import torchtext
from torchtext import data
from torchtext.data import Field, BucketIterator, TabularDataset
from sklearn.model_selection import train_test_split
import math
import pandas as pd
import copy
from datetime import datetime
from nltk.translate.bleu_score import sentence_bleu
# from janome.tokenizer import Tokenizer

if torch.cuda.is_available():
    device = 'cuda:0'
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = 'cpu'
print(f'Running on device: {device}')

Running on device: cuda:0


In [63]:
df = pd.read_csv('kurd-eng-full.csv')
df.head()

,Unnamed: 0,ku,en
0,0,بە ناوی خوای بەخشندەی میهرەبان .,"in the name of allah , most benevolent , ever-..."
1,1,‌سوپاس و ستایش هەر شایستەی خوایە و بۆ خوایە، ک...,"all praise be to allah , lord of all the worlds ,"
2,2,بەخشندەی میهرەبان‌، کانگای ڕەحمەت و میهرەبانیە .,"most beneficent , ever-merciful ,"
3,3,خاوەن و سەرداری ڕۆژی پاداشت و سزایە .,king of the day of judgement .
4,4,خوایە، تەنها هەر تۆ دەپەرستین و هەر لەتۆش داوا...,"you alone we worship , and to you alone turn f..."


In [19]:
BATCH_SIZE = 20
EPOCHS = 70

D_MODEL = 512
HEADS = 8
N = 6

#Tokenization using Spacy

In [20]:
JA = spacy.blank('fa')
EN = spacy.load("en_core_web_sm")

In [60]:
def tokenize_ku(sentence):
    return [tok.text for tok in JA.tokenizer(str(sentence))]

def tokenize_en(sentence):
    return [tok.text for tok in EN.tokenizer(str(sentence))]

#Creating Field Datatorch text

In [22]:
JA_TEXT = Field(tokenize=tokenize_ku) 
EN_TEXT = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>')

#Splitting the data set

In [23]:
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False) 
test.to_csv('test.csv', index=False)

In [24]:
data_fields = [('ku', JA_TEXT), ('en', EN_TEXT)]

train, val, test = TabularDataset.splits(path='./',
                        train='train.csv', 
                        validation='val.csv',
                        test = 'test.csv',
                        format='csv',
              
                        fields = data_fields )

#Creating Vocabulary dictionary

In [25]:

JA_TEXT.build_vocab(train, val) 
EN_TEXT.build_vocab(train, val)

In [26]:
train_iter = BucketIterator(
    train,
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.en),
    shuffle=True
)

In [27]:
batch = next(iter(train_iter)) 
print(batch.en)

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  14,    7,   19,  ...,   16,  319,   25],
        [2413,  274,   12,  ...,   23,  175, 4184],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])


In [28]:
global max_src_in_batch, max_tgt_in_batch

def batch_size_fn(new, count, sofar):
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.ku))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.en) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

In [29]:
train_iter = MyIterator(
    train,
    batch_size=3000,
    device=0,
    repeat=False,
    sort_key=lambda x: (len(x.ku), len(x.en)),
    batch_size_fn=batch_size_fn,
    train=True,
    shuffle=True
)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [30]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(Embedder, self).__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        
    def forward(self, x):
        return self.embed(x)

# create constant 'pe' matrix with values dependant on pos and i

In [31]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 512):
        super().__init__()
        self.d_model = d_model
        
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x * math.sqrt(self.d_model)
        seq_len = x.size(1)
        x = x + torch.autograd.Variable(self.pe[:,:seq_len], \
        requires_grad=False).to(device)
        return x

# creates mask with 0s wherever there is padding in the input

In [32]:
def create_masks(input_seq, target_seq):

    input_pad = JA_TEXT.vocab.stoi['<pad>']
    input_msk = (input_seq != input_pad).unsqueeze(1)
    
    target_pad = EN_TEXT.vocab.stoi['<pad>']
    target_msk = (target_seq != target_pad).unsqueeze(1)
    size = target_seq.size(1) # get seq_len for matrix
    nopeak_mask = np.triu(np.ones((1, size, size)), k=1).astype(np.uint8)
    nopeak_mask = torch.autograd.Variable(torch.from_numpy(nopeak_mask) == 0).to(device)
    target_msk = target_msk & nopeak_mask
    return input_msk, target_msk

In [33]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        bs = q.size(0)
        
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)

       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output

In [34]:
def attention(q, k, v, d_k, mask=None, dropout=None):

    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)

    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    scores = F.softmax(scores, dim=-1)

    if dropout is not None:
        scores = dropout(scores)

    output = torch.matmul(scores, v)

    return output

In [35]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):

        super().__init__() 

        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) 
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)

        return x

In [36]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):

        super().__init__()
    
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):

        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias

        return norm

In [37]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):

        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
 
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))

        return x

In [38]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):

        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model).to(device)

    def forward(self, x, e_outputs, src_mask, trg_mask):

            x2 = self.norm_1(x)
            x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
            x2 = self.norm_2(x)
            x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
            src_mask))
            x2 = self.norm_3(x)
            x = x + self.dropout_3(self.ff(x2))

            return x

In [39]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [40]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):

        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):

        x = self.embed(src)
        x = self.pe(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):

        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):

        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [41]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):

        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):

        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [42]:
src_vocab = len(JA_TEXT.vocab)
trg_vocab = len(EN_TEXT.vocab)

model = Transformer(src_vocab, trg_vocab, D_MODEL, N, HEADS)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [43]:
input_pad = JA_TEXT.vocab.stoi['<pad>']
target_pad = EN_TEXT.vocab.stoi['<pad>']

In [44]:
MultiHeadAttention(HEADS, D_MODEL)

MultiHeadAttention(
  (q_linear): Linear(in_features=512, out_features=512, bias=True)
  (v_linear): Linear(in_features=512, out_features=512, bias=True)
  (k_linear): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out): Linear(in_features=512, out_features=512, bias=True)
)

In [45]:
iters = []
for epoch in range(1):
  for i, batch in enumerate(train_iter):
    iters.append(i+1)
len(iters)

1004

In [46]:
def train_model(model, epochs, print_every=50):
    
    model.train()
    
    start = datetime.now()
    temp = start
    
    total_loss = 0
    
    for epoch in range(epochs):
       
        for i, batch in enumerate(train_iter):
            src = batch.ku.transpose(0, 1)
            trg = batch.en.transpose(0, 1)
            # the Kurdish sentence we input has all words except
            # the last, as it is using each word to predict the next
            
            trg_input = trg[:, :-1]
            
            # the words we are trying to predict
            
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above
            
            src_mask, trg_mask = create_masks(src, trg_input)
            
            preds = model(src, trg_input, src_mask, trg_mask)
            
            optim.zero_grad()
            
            loss = F.cross_entropy(
                preds.view(-1, preds.size(-1)),
                targets,
                ignore_index=target_pad
            )
            loss.backward()
            optim.step()
            
            total_loss += loss.item()
            Loss_hist = []
            Perplexity_hist = []
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                perplexity  = math.exp(loss_avg)
                print("time = {}, epoch {}, iter = {}, loss = {}, perplexity = {}, {} per {} iters".format(
                    (datetime.now() - start) // 60,
                    epoch + 1,
                    i + 1,
                    loss_avg,
                    perplexity,
                    datetime.now() - temp,
                    print_every
                ))
                if i+1 == 450:
                    Loss_hist.append(loss_avg)
                    Perplexity_hist.append(perplexity)
                total_loss = 0
                temp = datetime.now()
    
        print()

In [47]:
model.to(device)
train_model(model, EPOCHS)

time = 0:00:00.227568, epoch 1, iter = 50, loss = 9.119859247207641, perplexity = 9134.915761031612, 0:00:13.654142 per 50 iters
time = 0:00:00.446801, epoch 1, iter = 100, loss = 6.936057500839233, perplexity = 1028.706535045561, 0:00:13.153775 per 50 iters
time = 0:00:00.671047, epoch 1, iter = 150, loss = 6.374636135101318, perplexity = 586.7718864003892, 0:00:13.454571 per 50 iters
time = 0:00:00.889265, epoch 1, iter = 200, loss = 6.363563547134399, perplexity = 580.3106404422775, 0:00:13.092934 per 50 iters
time = 0:00:01.103558, epoch 1, iter = 250, loss = 6.178848066329956, perplexity = 482.4359019862641, 0:00:12.857426 per 50 iters
time = 0:00:01.318293, epoch 1, iter = 300, loss = 6.0575751781463625, perplexity = 427.3379610937259, 0:00:12.884011 per 50 iters
time = 0:00:01.537996, epoch 1, iter = 350, loss = 5.920366582870483, perplexity = 372.54825865721625, 0:00:13.182025 per 50 iters
time = 0:00:01.755900, epoch 1, iter = 400, loss = 5.852427549362183, perplexity = 348.07

In [48]:
def translate(model, src, max_len=80, custom_sentence=False):
    
    model.eval()
    
    if custom_sentence == True:
        src = tokenize_ku(src)
        src = torch.autograd.Variable(torch.LongTensor([[JA_TEXT.vocab.stoi[tok] for tok in src]])).to(device)
    
    src_mask = (src != input_pad).unsqueeze(-2)
    e_outputs = model.encoder(src, src_mask)

    outputs = torch.zeros(max_len).type_as(src.data)
    outputs[0] = torch.LongTensor([EN_TEXT.vocab.stoi['<sos>']])
    
    for i in range(1, max_len):    
        trg_mask = np.triu(np.ones((1, i, i)), k=1).astype('uint8')
        trg_mask = torch.autograd.Variable(torch.from_numpy(trg_mask) == 0).to(device)

        out = model.out(
            model.decoder(
                outputs[:i].unsqueeze(0),
                e_outputs,
                src_mask,
                trg_mask
            )
        )
        out = F.softmax(out, dim=-1)
        val, ix = out[:, -1].data.topk(1)

        outputs[i] = ix[0][0]
        if ix[0][0] == EN_TEXT.vocab.stoi['<eos>']:
            break

    return ' '.join([EN_TEXT.vocab.itos[ix] for ix in outputs[:i]])

In [49]:

import warnings
warnings.filterwarnings('ignore')

In [50]:

test_df = pd.read_csv('test.csv')

In [61]:
data_length = len(test_df["en"]);
total_score = 0;
print(data_length)
for i in range(data_length):
  print(i)
  reference = tokenize_en(test_df["en"][i]);
  candidate = tokenize_en(translate(model, test_df["ku"][i] , custom_sentence=True))
  candidate = candidate[3:]
  score = sentence_bleu(reference, candidate)
  total_score += score;

  print(total_score/data_length)

Streaming output truncated to the last 5000 lines.
37496
0.430764465033567
37497
0.4307776323599959
37498
0.43079249893559096
37499
0.430805424557002
37500
0.4308183501784131
37501
0.430832079063471
37502
0.43084478365263373
37503
0.43085665638699466
37504
0.43087086054949175
37505
0.4308845894345497
37506
0.4308956617921421
37507
0.43091009699239186
37508
0.43091009699239186
37509
0.430922697861947
37510
0.43093941804158936
37511
0.4309547892825372
37512
0.43096862419002424
37513
0.43098459936514794
37514
0.43099682366313863
37515
0.4310094245326938
37516
0.43102214649170817
37517
0.43103420213829796
37518
0.4310458817557029
37519
0.4310580202638464
37520
0.4310580202638464
37521
0.43107208020297
37522
0.4310848929260454
37523
0.4310848929260454
37524
0.4310848929260454
37525
0.43110121941383733
37526
0.43110121941383733
37527
0.43111767198425627
37528
0.43111767198425627
37529
0.431130376573419
37530
0.4311453216033242
37531
0.43115714455573195
37532
0.43117007017714304
37533
0.43117

#Bleu Score

In [62]:
print(f'BLEU score = {(total_score/data_length)*100}')

BLEU score = 45.95644686534556


In [52]:
translate(model, 'من تۆم خۆشدەوێت' , custom_sentence=True)

'<sos> i love you and i love you ,'

In [53]:
translate(model, 'من بۆ بازاڕ دەچم' , custom_sentence=True)

"<sos> i 'm going to go to the marketing game ,"

In [54]:
translate(model, 'ڕۆژێک لە ڕۆژان، نەخۆشیەکی ترسناک بوو بە هەڕەشە' , custom_sentence=True)

'<sos> once upon one time , there was a dread disease'

In [55]:
translate(model, 'دوێنێ ئەمریکا رایگەیاند، ئێران نایەوێت تۆمەتی خەمساردی لەدانوستانەکانی ڤیەننا بخاتە سەر شانی خۆی و رەتیشیکردەوە هیچ رێککەوتنێکیان لەبارەی ئاڵوگۆڕی زیندانیان کردبێت' , custom_sentence=True)

'<sos> the us ( associate - gods ever , iran was seen by iran and worked no keep back on the with no leaves of the become such as had any complete .'

In [56]:
translate(model, 'ئێوە لە کوێ بوون دوێنێ' , custom_sentence=True)

'<sos> where you were , like , yesterday ,'

In [57]:
translate(model, 'ئەوان دەڕۆن بۆ بازاڕ' , custom_sentence=True)

'<sos> they go in markets all market products'

In [58]:
translate(model, 'ناتوانم بچم بۆ ماڵی مامم' , custom_sentence=True)

"<sos> i ca n't go to my paternal house"

In [59]:
translate(model,'ئێوە هەمیشە لە دڵخۆشیدا بژین' , custom_sentence=True)

'<sos> you are always happiness .'